<a href="https://colab.research.google.com/github/m10923039/m10923039_hw01/blob/main/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Dense, Activation,Dropout

讀取檔案

In [2]:
df_data=pd.read_csv('/content/drive/MyDrive/ObesityDataSet_raw_and_data_sinthetic.csv')

# **資料前處理**

## lable encoding

## family_history_with_overweight 超重的家族病史(是/否) <br>
no  -> 0 <br>
yes  -> 1
## SMOKE 抽煙(是/否) <br>
no  -> 0 <br>
yes  -> 1 <br>
## SCC 卡路里消耗監視(是/否)
no  -> 0 <br>
yes  -> 1 <br>
## CAEC 兩餐之間的食物消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
## CALC 酒精消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
# one hot 欄位
Gender 性別<br>
MTRANS 交通工具<br>

In [3]:
# df_data
df_data.loc()[df_data['family_history_with_overweight']=='no','family_history_with_overweight']=0
df_data.loc()[df_data['family_history_with_overweight']=='yes','family_history_with_overweight']=1

df_data.loc()[df_data['FAVC']=='no','FAVC']=0
df_data.loc()[df_data['FAVC']=='yes','FAVC']=1

df_data.loc()[df_data['SMOKE']=='no','SMOKE']=0
df_data.loc()[df_data['SMOKE']=='yes','SMOKE']=1

df_data.loc()[df_data['SCC']=='no','SCC']=0
df_data.loc()[df_data['SCC']=='yes','SCC']=1

df_data.loc()[df_data['CAEC']=='No','CAEC']=1
df_data.loc()[df_data['CAEC']=='no','CAEC']=1
df_data.loc()[df_data['CAEC']=='Sometimes','CAEC']=2
df_data.loc()[df_data['CAEC']=='Frequently','CAEC']=3
df_data.loc()[df_data['CAEC']=='Always','CAEC']=4

df_data.loc()[df_data['CALC']=='no','CALC']=1
df_data.loc()[df_data['CALC']=='Sometimes','CALC']=2
df_data.loc()[df_data['CALC']=='Frequently','CALC']=3
df_data.loc()[df_data['CALC']=='Always','CALC']=4

df_data[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']] =df_data[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']].astype(int)

df_data.loc()[df_data['NObeyesdad']=='Insufficient_Weight','NObeyesdad']=1
df_data.loc()[df_data['NObeyesdad']=='Normal_Weight','NObeyesdad']=2
df_data.loc()[df_data['NObeyesdad']=='Obesity_Type_I','NObeyesdad']=3
df_data.loc()[df_data['NObeyesdad']=='Obesity_Type_II','NObeyesdad']=4
df_data.loc()[df_data['NObeyesdad']=='Obesity_Type_III','NObeyesdad']=5
df_data.loc()[df_data['NObeyesdad']=='Overweight_Level_I','NObeyesdad']=6
df_data.loc()[df_data['NObeyesdad']=='Overweight_Level_II','NObeyesdad']=7


In [ ]:
df_data

In [4]:
df_x=df_data.drop(['NObeyesdad'],axis=1)
df_y=df_data.loc[:,'NObeyesdad']

df_x = pd.get_dummies(df_x)
df_y = pd.get_dummies(df_y)

x_train,x_test,y_train,y_test = train_test_split(df_x,df_y,test_size=0.2)



In [ ]:
x_train.dtypes

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

In [5]:
input_layer = Input(shape=(x_train.shape[1],))
dense_layer_1 =Dense(32,kernel_initializer=tf.keras.initializers.GlorotNormal() ,activation='relu', name ="nl1" )(input_layer)
dense_layer_2 = Dense(16,kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='relu', name ="nl2" )(dense_layer_1)
dense_layer_2 = Dense(10,activation='relu', name ="nl3" )(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl4" )(dense_layer_2)
dense_layer_2 = Dropout(0.5)(dense_layer_2)
output = Dense(y_train.shape[1],activation='softmax')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output)
print(model.summary())


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
nl1 (Dense)                  (None, 32)                704       
_________________________________________________________________
nl2 (Dense)                  (None, 16)                528       
_________________________________________________________________
nl3 (Dense)                  (None, 10)                170       
_________________________________________________________________
nl4 (Dense)                  (None, 10)                110       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 77    

In [12]:
# model.reset_metrics()
# model.reset_states()

callback = tf.keras.callbacks.EarlyStopping(monitor='precision_3',mode='max', patience=15 ,restore_best_weights=True)
Adam = tf.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.997, epsilon=1e-08, decay=0.0)
model.compile(optimizer=Adam, 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
              )

history = model.fit(x=x_train, y=y_train,  epochs=50,batch_size =3, callbacks=[callback] )

Epoch 1/50
563/563 [==============================] - 2s 2ms/step - loss: 0.9212 - precision_3: 0.7967 - recall_3: 0.4427
Epoch 2/50
563/563 [==============================] - 1s 2ms/step - loss: 0.9204 - precision_3: 0.7726 - recall_3: 0.3997
Epoch 3/50
563/563 [==============================] - 1s 2ms/step - loss: 0.9017 - precision_3: 0.7699 - recall_3: 0.4177
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 0.8797 - precision_3: 0.8108 - recall_3: 0.4508
Epoch 5/50
563/563 [==============================] - 1s 2ms/step - loss: 0.8804 - precision_3: 0.7761 - recall_3: 0.4366
Epoch 6/50
563/563 [==============================] - 1s 2ms/step - loss: 0.8857 - precision_3: 0.7842 - recall_3: 0.4273
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 0.8627 - precision_3: 0.7780 - recall_3: 0.4228
Epoch 8/50
563/563 [==============================] - 1s 2ms/step - loss: 0.8702 - precision_3: 0.7915 - recall_3: 0.4196
Epoch 9/50
563/563 [====

**指標F1**

In [13]:
preds = model.evaluate(x = x_test, y =y_test )

F1 = 2 * (preds[1] * preds[2]) / (preds[1] + preds[2])
print('Precision', preds[1])
print('Recall}', preds[2])
print('F1',F1)


14/14 [==============================] - 0s 1ms/step - loss: 0.6254 - precision_3: 0.8328 - recall_3: 0.6241
Precision 0.8328076004981995
Recall} 0.6241135001182556
F1 0.7135135406469062


In [ ]:

# pd.DataFrame(history.history).plot(figsize = (8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 2.5)
# plt.show()


In [ ]:
y_predict=model.predict(x_test)


In [ ]:
y_predict

In [ ]:
# y_predict[y_predict.max(axis=1)]=y_predict.max(axis=1)
# arr.max(axis=1)
np.argmax(y_predict[0])
np.argmax(y_test[0])